In [1]:
import pandas as pd
import requests
import json

In [2]:
## Generate access token

## Click on the following link to get the "Code" parameter for the post method to get a new Token
## Example: https://mysite.com/?code=TG-xxxxxxxxxx-xxxxxxxx -> In "params" copy the code generated that starts in "TG-... in the "code" parameter

'https://auth.mercadolibre.com.co/authorization?response_type=code&client_id=2461416230553732&redirect_uri=https://mysite.com'

 
params = {'grant_type': "authorization_code", "client_id": "2461416230553732", 
          "client_secret": "6Wc0kqb5cw0nTUoJo0Y3Ws1GGalDB2Dk", "code": "TG-65ea94be6f87530001172ec8-588943530" , 
          "redirect_uri": "https://mysite.com"}
request = requests.post('https://api.mercadolibre.com/oauth/token', data = params)
token = request.json().get('access_token')
token

'APP_USR-2461416230553732-030800-314bfafc9078e1eec52b37cdbe1f38a3-588943530'

In [3]:
## Get all the User Ids from a given category and country. In this case we're going to select Colombia (Site = 'MCO') and all available categories
## Write a while statement to loop through the results

results = []
offset = 0

cats = requests.get('https://api.mercadolibre.com/sites/MCO/categories')
resp = json.loads(cats.content)
categories = pd.DataFrame(resp)

while offset < 1000:
    for value in categories['id']:
        url = f'https://api.mercadolibre.com/sites/MCO/search?category={value}&offset={offset}'
        request = requests.get(url)
        items = request.json().get('results')
        results.append(items)
    offset+=50

In [4]:
#Code block that shows all the unique User ids and their Categories

seller_ids = []

for inner_list in results:
    for item in inner_list:
        seller_name = item.get("seller")
        id = seller_name.get("id")
        seller_category = item.get("category_id")

        seller_ids.append([id,seller_category])

sell_cat = pd.DataFrame(seller_ids, columns= ['seller_id', 'category_id'])
sell_cat = sell_cat.drop_duplicates()

In [8]:
## Using the Ids list, loop through the seller info to get the attributes and metrics related to the seller

token_comp = "Bearer " + token
header = {"Authorization" : token_comp }

seller_info = []

for i in sell_cat['seller_id']:
    user_info = requests.get(f'https://api.mercadolibre.com//sites/MCO/search?seller_id={i}', headers=header)
    seller_info.append(user_info.json().get('seller'))

In [9]:
## Put together the obtained information in a Dataframe for data manipulation purposes. Since there are nested JSONs, extraction of this data is required to get the final dataset.

df = pd.DataFrame(seller_info)

def extract_lvl1(df):
  
  df["level"] = df["seller_reputation"].apply(lambda x: x["level_id"])
  df["seller_status"] = df["seller_reputation"].apply(lambda x: x["power_seller_status"])
  df["transactions"] = df["seller_reputation"].apply(lambda x: x["transactions"])
  df["metrics"] = df["seller_reputation"].apply(lambda x: x["metrics"])
  return df

def extract_lvl2(df):
  extract_lvl1(df)
  df["canceled_orders"] = df["transactions"].apply(lambda x: x["canceled"])
  df["ratings"] = df["transactions"].apply(lambda x: x["ratings"])
  df["total_orders"] = df["transactions"].apply(lambda x: x["total"])
  df["sales"] = df["metrics"].apply(lambda x: x["sales"])
  df["claims"] =                               df["metrics"].apply(lambda x: x["claims"])
  df["delayed_handling_time"] = df["metrics"].apply(lambda x: x["delayed_handling_time"])
  df["cancellations"] = df["metrics"].apply(lambda x: x["cancellations"])
  
  return df

def extract_lvl3(df):
    extract_lvl2(df)
    df['negative_rating'] = df["ratings"].apply(lambda x: x["negative"])
    df['neutral_rating'] = df["ratings"].apply(lambda x: x["neutral"])
    df['positive_rating'] = df["ratings"].apply(lambda x: x["positive"])
    df['completed_sales'] = df["sales"].apply(lambda x: x["completed"])
    df['claims'] = df["claims"].apply(lambda x: x["rate"])
    df['delayed_time'] = df["delayed_handling_time"].apply(lambda x: x["rate"])
    df['cancellations'] = df["cancellations"].apply(lambda x: x["rate"])

    return df

df = extract_lvl3(df)

df = df[['id','nickname','registration_date','tags','level','seller_status','canceled_orders','total_orders','claims','cancellations','negative_rating','neutral_rating','positive_rating','completed_sales','delayed_time']]

final_data = pd.merge(df, sell_cat, how='left', left_on= 'id' , right_on= 'seller_id')

final_data = final_data[['id','category_id','nickname','registration_date','tags','level','seller_status','canceled_orders','total_orders','claims','cancellations','negative_rating','neutral_rating','positive_rating','completed_sales','delayed_time']]
##final_data.to_csv('C:/Users/migca/Desktop/Dataset.csv', index=False)
final_data


,id,category_id,nickname,registration_date,tags,level,seller_status,canceled_orders,total_orders,claims,cancellations,negative_rating,neutral_rating,positive_rating,completed_sales,delayed_time
0,42485793,MCO90069,MOBILEK2011,2011-01-20T18:09:40.000-04:00,"[normal, eshop, messages_as_seller]",5_green,platinum,360,4329,0.0153,0.0063,0.02,0.02,0.96,1025,0.0197
1,160820235,MCO407376,BUILDER CO,2014-06-14T19:51:26.000-04:00,"[normal, mshops, messages_as_seller, eshop]",5_green,platinum,3750,103264,0.0035,0.0019,0.14,0.04,0.82,26428,0.0009
2,160820235,MCO1667,BUILDER CO,2014-06-14T19:51:26.000-04:00,"[normal, mshops, messages_as_seller, eshop]",5_green,platinum,3750,103264,0.0035,0.0019,0.14,0.04,0.82,26428,0.0009
3,160820235,MCO177118,BUILDER CO,2014-06-14T19:51:26.000-04:00,"[normal, mshops, messages_as_seller, eshop]",5_green,platinum,3750,103264,0.0035,0.0019,0.14,0.04,0.82,26428,0.0009
4,160820235,MCO179924,BUILDER CO,2014-06-14T19:51:26.000-04:00,"[normal, mshops, messages_as_seller, eshop]",5_green,platinum,3750,103264,0.0035,0.0019,0.14,0.04,0.82,26428,0.0009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126229,23832163,MCO3530,NHVIVAS,2010-02-13T19:58:24.000-04:00,"[normal, mshops, messages_as_seller]",5_green,None,4,48,0.0000,0.0208,0.01,0.00,0.99,44,0.0714
126230,58226416,MCO3530,EGECOL SAS,2011-06-06T20:51:16.000-04:00,"[normal, mshops, messages_as_seller, eshop]",5_green,None,11,202,0.0099,0.0049,0.05,0.03,0.92,191,0.0326
126231,229013484,MCO172908,JUANANDRESMONTENEGRORETAVIZC,2016-09-20T19:07:55.000-04:00,"[normal, messages_as_seller]",5_green,None,16,287,0.0090,0.0000,0.00,0.00,1.00,102,0.0943
126232,512969649,MCO412104,AINCRADSTORE,2020-01-11T12:40:00.000-04:00,"[normal, messages_as_seller, eshop]",4_light_green,None,3,24,0.0000,0.0000,0.00,0.04,0.96,21,0.0000
